# Huấn luyện mô hình

In [ ]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage.io import imread
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import cv2
import os
import glob
from PIL import Image # This will be used to read/modify images (can be done via OpenCV too)
from numpy import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Sử dụng HOG để rút trích đặc trưng


In [ ]:
# define parameters of HOG feature extraction
orientations = 8
pixels_per_cell = (6, 6)
cells_per_block = (2, 2)
threshold = .3

In [ ]:
# define path to images:
pos_im_path = r"/content/drive/MyDrive/Self_LearnCV/Dataset/train/ped" # This is the path of our positive input dataset
#Others
pos_im_path_added = r"/content/drive/MyDrive/Self_LearnCV/Dataset/test/ped"
# define the same for negatives
neg_im_path= r"/content/drive/MyDrive/Self_LearnCV/Dataset/train/bkg"


In [ ]:
# read the image files:
pos_im_listing = os.listdir(pos_im_path) # it will read all the files in the positive image path (so all the required images)
neg_im_listing = os.listdir(neg_im_path)
# read the image files:
pos_im_listing_added = os.listdir(pos_im_path_added) # it will read all the files in the positive image path (so all the required images)
#neg_im_listing_test = os.listdir(neg_im_path_test)
num_pos_samples = size(pos_im_listing_added) # simply states the total no. of images
num_neg_samples = size(neg_im_listing)

500
2510


In [ ]:
data= []
labels = []
# compute HOG features and label them:

In [ ]:
for file in pos_im_listing: #this loop enables reading the files in the pos_im_listing variable one by one
    img = Image.open(pos_im_path + '/' + file) # open the file
    img = img.resize((64,128))
    gray = img.convert('L') # convert the image into single channel i.e. RGB to grayscale
    # calculate HOG for positive features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    data.append(fd)
    labels.append(1)

In [ ]:
for file in pos_im_listing_added: #this loop enables reading the files in the pos_im_listing_added variable one by one
    img = Image.open(pos_im_path_added + '/' + file) # open the file
    img = img.resize((64,128))
    gray = img.convert('L') # convert the image into single channel i.e. RGB to grayscale
    # calculate HOG for positive features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    data.append(fd)
    labels.append(1)

In [ ]:
# Same for the negative images
for file in neg_im_listing:
    img= Image.open(neg_im_path + '/' + file)
    img = img.resize((64,128))
    gray= img.convert('L')
    # Now we calculate the HOG for negative features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)
    data.append(fd)
    labels.append(0)

In [ ]:
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
# Partitioning the data into training and testing splits, using 80%
# of the data for training and the remaining 20% for testing
print(" Constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(
	np.array(data), labels, test_size=0.20, random_state=42)

 Constructing training/testing split...


## Phân loại 2 lớp với mô hình SVM

In [ ]:
from sklearn import svm

In [ ]:
clf2 = svm.LinearSVC(C=0.01, max_iter=1000, class_weight='balanced', verbose = 1)

In [ ]:
#%% Train the linear SVM
print(" Training Linear SVM classifier...")
clf2.fit(trainData, trainLabels)

 Training Linear SVM classifier...
[LibLinear]

LinearSVC(C=0.01, class_weight='balanced', verbose=1)

In [ ]:
#%% Evaluate the classifier
print(" Evaluating classifier on test data ...")
predictions = clf2.predict(testData)
print(classification_report(testLabels, predictions))

 Evaluating classifier on test data ...
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       492
           1       0.98      0.97      0.98       612

    accuracy                           0.97      1104
   macro avg       0.97      0.97      0.97      1104
weighted avg       0.97      0.97      0.97      1104



In [ ]:
# Save the model:
joblib.dump(clf2, '/content/drive/MyDrive/Self_LearnCV/model.npy')

['/content/drive/MyDrive/Self_LearnCV/model.npy']

## Nhận diện người đi bộ bằng kết hợp HOG, SVM và các kỹ thuật thị giác máy tính cơ bản

### Sliding window và pyramid image


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
#%%
# Upload the saved svm model:
model = joblib.load('/content/drive/MyDrive/Self_LearnCV/model.npy')

In [ ]:
#Define HOG Parameters
# change them if necessary to orientations = 8, pixels per cell = (16,16), cells per block to (1,1) for weaker HOG
orientations = 8
pixels_per_cell = (6, 6)
cells_per_block = (2, 2)
threshold = .3

In [ ]:
# define the sliding window:
def sliding_window(image, stepSize, windowSize):# image is the input, step size is the no.of pixels needed to skip and windowSize is the size of the actual window
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):# this line and the line below actually defines the sliding part and loops over the x and y coordinates
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])

In [ ]:
# Test the trained classifier on an image below!
scale = 0
detections = []

In [ ]:
# defining the size of the sliding window (has to be, same as the size of the image in the training data)
def get_objects(img):
  scale = 0
  detections = []
  (winW, winH)= (64,128)
  windowSize=(winW,winH)
  downscale=1.5
  # Apply sliding window:
  for resized in pyramid_gaussian(img, downscale=1.5): # loop over each layer of the image that you take!
      # loop over the sliding window for each layer of the pyramid
      for (x,y,window) in sliding_window(resized, stepSize=5, windowSize=(winW,winH)):
          # if the window does not meet our desired window size, ignore it!
          if window.shape[0] != winH or window.shape[1] !=winW: # ensure the sliding window has met the minimum size requirement
              continue
          #window.shape
          #if window.ndim == 2:
          #  window = window[:, :, np.newaxis]
          #window=color.rgb2gray(window)
          #window.shape
          fds = hog(window, orientations, pixels_per_cell, cells_per_block, block_norm='L2')  # extract HOG features from the window captured
          fds = fds.reshape(1, -1) # reshape the image to make a silouhette of hog
          pred = model.predict(fds) # use the SVM model to make a prediction on the HOG features extracted from the window

          if pred == 1:
              if model.decision_function(fds) > 0.6:  # set a threshold value for the SVM prediction i.e. only firm the predictions above probability of 0.6
                  #print("Detection:: Location -> ({}, {})".format(x, y))
                  #print("Scale ->  {} | Confidence Score {} \n".format(scale,model.decision_function(fds)))
                  detections.append((int(x * (downscale**scale)), int(y * (downscale**scale)), model.decision_function(fds),
                                    int(windowSize[0]*(downscale**scale)), # create a list of all the predictions found
                                        int(windowSize[1]*(downscale**scale))))
      scale+=1

  clone = resized.copy()
  return detections

In [ ]:
def get_nms_bb(detections, img):
  for (x_tl, y_tl, _, w, h) in detections:
      cv2.rectangle(img, (x_tl, y_tl), (x_tl + w, y_tl + h), (0, 0, 255), thickness = 2)
  rects = np.array([[x, y, x + w, y + h] for (x, y, _, w, h) in detections]) # do nms on the detected bounding boxes
  sc = [score[0] for (x, y, score, w, h) in detections]
  #print("detection confidence score: ", sc)
  sc = np.array(sc)
  predicted_bb = non_max_suppression(rects, probs = sc, overlapThresh = 0.2)
  print("detection utmost confidence score: ", predicted_bb)
  return predicted_bb

### Thay đổi kích thước các bounding_box để phù hợp với ảnh gốc
*Vì trước đó, kích thước ảnh bị resize (200,300)*

In [ ]:
# scaling each predicted bouding box
def rescale_bounding_box(bounding_box, scaling_factor):
    xmin, ymin, xmax, ymax = bounding_box

    # Rescale the bounding box coordinates
    rescaled_xmin = int(xmin / scaling_factor[1])
    rescaled_ymin = int(ymin / scaling_factor[0])
    rescaled_xmax = int(xmax / scaling_factor[1])
    rescaled_ymax = int(ymax / scaling_factor[0])

    rescaled_bounding_box = (rescaled_xmin, rescaled_ymin, rescaled_xmax, rescaled_ymax)
    return rescaled_bounding_box

In [ ]:
# scaling the whole set of predicted bb
def rescaled_bounding_boxes(bounding_boxes, scaling_factor):
    # Rescale the bounding boxes
    rescaled_bounding_boxes = []
    for bbox in bounding_boxes:
        rescaled_bbox = rescale_bounding_box(bbox, scaling_factor)
        rescaled_bounding_boxes.append(rescaled_bbox)

    return rescaled_bounding_boxes

In [ ]:
def calculate_scaling_factor(original_width, original_height, current_width, current_height):
    scaling_factor_width = current_width / original_width
    scaling_factor_height = current_height / original_height
    scaling_factor = (scaling_factor_width, scaling_factor_height)
    return scaling_factor

### Ground truth từ annotation


In [ ]:
import re

In [ ]:
def read_bounding_boxes_from_file(file_path):
    bounding_boxes = []

    with open(file_path, 'r') as file:
        content = file.read()

    # Extract bounding box information using regular expressions
    pattern = r'Bounding box for object \d+ "(\w+)" \(Xmin, Ymin\) - \(Xmax, Ymax\) : \((\d+), (\d+)\) - \((\d+), (\d+)\)'
    matches = re.findall(pattern, content)

    for match in matches:
        label = match[0]
        xmin, ymin, xmax, ymax = map(int, match[1:])

        # Store bounding box information in a dictionary
        bounding_box = []
        bounding_box.append(xmin)
        bounding_box.append(ymin)
        bounding_box.append(xmax)
        bounding_box.append(ymax)

        # Add the bounding box to the list
        bounding_boxes.append(bounding_box)

    return bounding_boxes

### Đánh giá phương pháp với Precision và Recall

#### IoU


In [ ]:
def get_iou(ground_truth, pred):
    # coordinates of the area of intersection.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])

    # Intersection height and width.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))

    area_of_intersection = i_height * i_width

    # Ground Truth dimensions.
    gt_height = ground_truth[3] - ground_truth[1] + 1
    gt_width = ground_truth[2] - ground_truth[0] + 1

    # Prediction dimensions.
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1

    area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection

    iou = area_of_intersection / area_of_union

    return iou

#### Từ IoU ta tìm TP, FP, FN

In [ ]:
def Get(predicted_boxes, ground_truth_boxes, iou_threshold = 0.5):
  true_positives = 0
  false_positives = 0
  false_negatives = 0

  for pred_box in predicted_boxes:
      match_found = False

      for gt_box in ground_truth_boxes:
          iou = get_iou(pred_box, gt_box)

          if iou >= iou_threshold :
              match_found = True
              break

      if match_found:
          true_positives += 1
      else:
          false_positives += 1

  false_negatives = len(ground_truth_boxes) - true_positives

  return (true_positives, false_positives, false_negatives)


#### Kết quả đánh giá



In [ ]:
test_image_pth = r"/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/PNGImages"
test_anno_pth = r"/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/Annotation"

In [ ]:
test_im_listing = sorted(os.listdir(test_image_pth))
test_anno_listing = sorted(os.listdir(test_anno_pth))

In [ ]:
#IoUs = []
TP = 0
FP = 0
FN = 0

In [ ]:
for (img_file, anno_file) in zip(test_im_listing, test_anno_listing): #this loop enables reading the files in the pos_im_listing variable one by one
    #img = Image.open(test_image_pth + '/' + file) # open the file
    img = os.path.join(test_image_pth, img_file)

    #img = '/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/PNGImages/PennPed00001.png'
    image = cv2.imread(img)
    gray = cv2.imread(img, 0)
    gray = cv2.resize(gray,(300,200))
    #cv2_imshow(gray)

    print(img)

    detections = get_objects(gray)
    pick = get_nms_bb(detections, gray)
    #print(pick)

    # Saving the ground_truth
    anno = os.path.join(test_anno_pth, anno_file)
    print(anno)
    ground_truth = read_bounding_boxes_from_file(anno)
    print(ground_truth)

    # scaling bouding box
    scaling_factor = calculate_scaling_factor(image.shape[0], image.shape[1], gray.shape[0], gray.shape[1])
    rescaled = rescaled_bounding_boxes(pick, scaling_factor)

    #Couting TP, FP, FN:
    true_positives, false_positives, false_negatives = Get(rescaled, ground_truth, iou_threshold = 0.5)
    TP +=true_positives
    FP +=false_positives
    FN +=false_negatives

/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/PNGImages/FudanPed00001.png
detection utmost confidence score:  []
/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/Annotation/FudanPed00001.txt
[[160, 182, 302, 431], [420, 171, 535, 486]]
/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/PNGImages/FudanPed00002.png
detection utmost confidence score:  [[ 60  65 124 193]]
/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/Annotation/FudanPed00002.txt
[[68, 93, 191, 380]]
/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/PNGImages/FudanPed00003.png
detection utmost confidence score:  [[200  65 264 193]]
/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/Annotation/FudanPed00003.txt
[[293, 135, 447, 421]]
/content/drive/MyDrive/Self_LearnCV/Data/Pedestrian/PennFudanPed/PNGImages/FudanPed00004.png
detection utmost confidence score:  [[140  35 204 163]]
/content/drive/MyDrive/Self_LearnCV/Data/Pedestria

In [ ]:
precision = TP / float(TP + FP)
recall = TP / float(TP + FN)

In [ ]:
print("Precision:", precision)
print("Recall:", recall)

precision 0.614406779661017
recall 0.34278959810874704
